# NFM

<img src="../img/nfm.png" width="600">

In [1]:
import pandas as pd
import numpy as np
import json
import torch
from torch import nn
from torch.utils.data.dataset import Dataset
from collections import OrderedDict

In [2]:
learning_rate = 0.001
batch_size = 64
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
with open('../data/movielens/feature_map.json') as obj:
    feature_map = json.load(obj)
feature_map

{'dataset_id': 'movielens',
 'num_fields': 26,
 'feature_specs': {'movieId': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 935,
   'index': 0},
  'userId': {'source': 'user',
   'type': 'categorical',
   'vocab_size': 22540,
   'index': 1},
  'rating': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 2},
  'timestamp': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 3},
  'releaseYear': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 4},
  'movieGenre1': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 5},
  'movieGenre2': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 6},
  'movieGenre3': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 15,
   'index': 7},
  'movieRatingCount': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 8},
  'movieAvgRating': {'source': 'item',
   'type': 'numerical',
 

In [4]:
class MovielensDataset(Dataset):
    def __init__(self, url):
        self.df = pd.read_csv(url)
    
    def __getitem__(self, idx):
        x, y = self.df.iloc[idx, :-1].values.astype(np.float32), self.df.iloc[idx, -1].astype(np.float32)
        return x, y
    
    def __len__(self):
        return self.df.shape[0]

In [5]:
train_dataset = MovielensDataset('../data/movielens/data_for_train.csv')
test_dataset = MovielensDataset('../data/movielens/data_for_test.csv')

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
class NFM(nn.Module):
    def __init__(self,
                 feature_map,
                 embedding_dim=10,
                 hidden_units=[256, 128, 64]):
        super(NFM, self).__init__()
        self.feature_map = feature_map
        # Embedding
        self.embedding = nn.ModuleDict()
        for feature, feature_spec in feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                self.embedding[feature] = nn.Linear(1, embedding_dim, bias=False)
            elif feature_spec['type'] == 'categorical':
                padding_idx = feature_spec.get('padding_idx', None)
                self.embedding[feature] = nn.Embedding(feature_spec['vocab_size'],
                                                       embedding_dim,
                                                       padding_idx=padding_idx)
        # LR
        self.batch_norm = nn.BatchNorm1d(feature_map['num_fields'])
        self.lr_layer = nn.Linear(feature_map['num_fields'], 1)
        # B-Interaction Layer
        self.bi_interaction_layer = BiInteractionPooling()
        # DNN
        hidden_units = [embedding_dim] + hidden_units
        hidden_layers = []
        for i in range(len(hidden_units) - 1):
            hidden_layers.append(nn.Linear(hidden_units[i], hidden_units[i + 1]))
            hidden_layers.append(nn.ReLU())
        hidden_layers.append(nn.Linear(hidden_units[-1], 1))
        self.dense_layer = nn.Sequential(*hidden_layers)
        # Sigmoid
        self.output_activation = nn.Sigmoid()

    def forward(self, X):
        feature_emb_list = []
        for feature, feature_spec in self.feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                raw_feature = X[:, feature_spec['index']].float().view(-1, 1)
            elif feature_spec['type'] == 'categorical':
                raw_feature = X[:, feature_spec['index']].long()
            embedding_vec = self.embedding[feature](raw_feature)
            feature_emb_list.append(embedding_vec)
        feature_emb = torch.stack(feature_emb_list, dim=1)
        out = self.lr_layer(self.batch_norm(X))
        bi_pooling_vec = self.bi_interaction_layer(feature_emb)
        out += self.dense_layer(bi_pooling_vec)
        y_pred = self.output_activation(out).squeeze(1)
        return y_pred

class BiInteractionPooling(nn.Module):
    def __init__(self):
        super(BiInteractionPooling, self).__init__()
        
    def forward(self, X):
        sum_of_square = torch.sum(X, dim=1) ** 2 # sum then square
        square_of_sum = torch.sum(X ** 2, dim=1) # square then sum
        bi_interaction = (sum_of_square - square_of_sum) * 0.5
        return bi_interaction

In [8]:
model = NFM(feature_map).to(device)
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)

        # Forward pass
        output = model(X)
        loss = criterion(output, y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 300 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(
                epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [300/1388] Loss: 0.3969
Epoch [1/5], Step [600/1388] Loss: 0.3406
Epoch [1/5], Step [900/1388] Loss: 0.1842
Epoch [1/5], Step [1200/1388] Loss: 0.2566
Epoch [2/5], Step [300/1388] Loss: 0.1183
Epoch [2/5], Step [600/1388] Loss: 0.0979
Epoch [2/5], Step [900/1388] Loss: 0.0565
Epoch [2/5], Step [1200/1388] Loss: 0.0461
Epoch [3/5], Step [300/1388] Loss: 0.0659
Epoch [3/5], Step [600/1388] Loss: 0.0717
Epoch [3/5], Step [900/1388] Loss: 0.0213
Epoch [3/5], Step [1200/1388] Loss: 0.0241
Epoch [4/5], Step [300/1388] Loss: 0.0274
Epoch [4/5], Step [600/1388] Loss: 0.0214
Epoch [4/5], Step [900/1388] Loss: 0.0359
Epoch [4/5], Step [1200/1388] Loss: 0.0100
Epoch [5/5], Step [300/1388] Loss: 0.0128
Epoch [5/5], Step [600/1388] Loss: 0.0249
Epoch [5/5], Step [900/1388] Loss: 0.0114
Epoch [5/5], Step [1200/1388] Loss: 0.0040


In [9]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device).bool()
        output = model(X)
        y_pred = output > 0.5
        total += y.shape[0]
        correct += (y_pred == y).sum().item()

    print('Accuracy of the model on the test images: {:.2f} %'.format(
        100 * correct / total))

Accuracy of the model on the test images: 100.00 %
